In [ ]:
import sys
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras import optimizers
from keras.optimizers import Adam, SGD
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from sklearn.model_selection import train_test_split
from fastai_utils import *

from collections import Counter
import pandas as pd
import numpy as np
import glob
import h5py
import cv2

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from datetime import datetime
import time

os.chdir('/data/Affinebkp150319/Affine')
from imgaug import augmenters as iaa
import imgaug as ia

start = time.time()

In [ ]:
os.chdir('/data/data_backup_affine/')

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#config = tf.ConfigProto(device_count = {'GPU':0})
#sess = tf.Session(config=config)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
n_epochs = 17
bs = 32
img_size = 224

model_split_1 = 'res4a_branch2a'
model_split_2 = 'fc_start'

date = datetime.now().strftime("%m%d%Y")
counter = str(int(np.round(start)))
model_name = 'resnet50_790split_6_class_SGD_224size_notrain3_conv_dropout0.2_clr_bn_imgaug_' + counter
print(model_name)

train_data_path = './Data_v2/final_train/'
validation_data_path = './Data_v2/final_validation/'
# test_data_path = './Data_v2/test/'
data_annotation_csv_path = './Data_v2/'
model_weights_no_top = './models/model_weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
save_model_path = './models/iterations/'+ date +'/' + model_name + '/'
tf_log_path = './tf-log/new_data/'

if not os.path.exists('./models/iterations/'+ date +'/'):
    os.mkdir('./models/iterations/'+ date +'/')
    
print(date)

In [ ]:
from keras import backend as K

classes_2 = [1, 2, 3, 6, 8, 9, 10, 11, 12]
classes_3 = ['F', 'L', 'R']
classes_4 = ['B', 'C', 'D', 'F', 'L', 'P', 'R', 'Y', 'Z']
# classes_4 = ['B', 'D', 'L', 'R', 'Y', 'Z']
classes_5 = ['A', 'E', 'L', 'M']
classes_6 = ['E', 'N', 'S', 'W']
classes_7 = [1, 2, 3, 4, 5, 6, 8, 9]
# classes_7 = [1, 2, 3]

# classes = [classes_2, classes_3, classes_4, classes_5, classes_6, classes_7]
classes = [classes_3, classes_4, classes_5, classes_6, classes_7]

tot_num_classes = np.sum(len(c) for c in classes)

split_idx = [np.sum(len(classes[j]) for j in range(i)) for i in range(len(classes))][1:]
print(split_idx)

# loss_wt = {}
# # for i in range(len(classes)):
# #     loss_wt['col'+str(i+2)] = np.round(len(classes[i])/tot_num_classes, 3)

# losses = [1,3,1,1,1,3]
# loss_wt = dict(zip(['col'+str(i+2) for i in range(len(classes))], losses))
# print(loss_wt)

In [ ]:
data = pd.read_csv(data_annotation_csv_path + 'Data_Annotation_v6.csv', encoding = 'latin-1')

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight

def one_hot_categorical(x, classes):
    
    lb_enc = LabelEncoder().fit(np.array(classes).reshape(-1,1))
    label_encoded = lb_enc.transform(x)
    
    recip_freq  = len(label_encoded)/ (len(lb_enc.classes_) * np.bincount(label_encoded).astype(np.float64))
    recip_freq[recip_freq == np.inf] = 0
    weight = recip_freq[lb_enc.transform(lb_enc.classes_)]    
    cw_dict = dict(zip(lb_enc.transform(lb_enc.classes_),np.round(weight,4)))
    
    enc = OneHotEncoder(sparse=False).fit(lb_enc.transform(np.array(classes).reshape(-1,1)).reshape(-1,1))
    enc.transform(label_encoded.reshape(-1,1))
    return enc.transform(label_encoded.reshape(-1,1)), cw_dict, lb_enc

y_2, cw_2, lb_2 = one_hot_categorical(data['Col2'],classes_2)    
y_3, cw_3, lb_3  = one_hot_categorical(data['Col3'],classes_3)
# Including only side crash code 
# Not including R, C, L
y_4, cw_4, lb_4  = one_hot_categorical(data['Col4'],classes_4)
y_5, cw_5, lb_5  = one_hot_categorical(data['Col5'],classes_5)
y_6, cw_6, lb_6  = one_hot_categorical(data['Col6'],classes_6)
y_7, cw_7, lb_7  = one_hot_categorical(data['Col7'],classes_7)

# lb = [lb_2, lb_3, lb_4, lb_5, lb_6, lb_7]
lb = [lb_3, lb_4, lb_5, lb_6, lb_7]

# cw = dict(zip(['col'+str(i+2) for i in range(len(classes))], [cw_2, cw_3, cw_4, cw_5, cw_6, cw_7]))
# print(cw)

# Create multi-label targets
# targets = np.concatenate((y_2, y_3, y_4, y_5, y_6, y_7), axis = 1)
targets = np.concatenate((y_3, y_4, y_5, y_6, y_7), axis = 1)

image_files = pd.DataFrame(targets)
image_files['filename'] = data['filename'].apply(lambda x: x+'.jpg')
print()
print('Found ' + str(image_files.shape[0]) + ' annotated images')
print(image_files.shape)

In [ ]:
train_ids = [i.split('/')[-1].strip() for i in glob.glob(train_data_path+'*.jpg')]
train_df = image_files.loc[image_files['filename'].isin(train_ids),:].drop_duplicates().reset_index(drop = True)
train_ids = list(train_df['filename'])
print(len(train_ids), train_df.shape)

val_ids = [i.split('/')[-1].strip() for i in glob.glob(validation_data_path+'*.jpg')]
val_df = image_files.loc[image_files['filename'].isin(val_ids),:].drop_duplicates().reset_index(drop = True)
val_ids = list(val_df['filename'])
print(len(val_ids), val_df.shape)

# test_ids = [i.split('/')[-1].strip() for i in glob.glob(test_data_path+'*.jpg')]
# test_df = image_files.loc[image_files['filename'].isin(test_ids),:].reset_index(drop = True).drop_duplicates()
# print(len(test_ids), test_df.shape)

In [ ]:
def split_outputs(generator, idx):
    while True:
        data = next(generator)
        x = data[0]
        y = np.split(data[1], idx, axis=1)
        yield x, y
        
def read_data(img_data_gen, base_dir, in_df, idx, path_col = 'filename',
                        y_col = 'targets', batch_size = 8, n_classes = 38, size = 512):
    
    print(in_df.shape)
    df_gen = img_data_gen.flow_from_dataframe(in_df, base_dir,
                                              x_col = 'filename', y_col= list(range(n_classes)),
                                              has_ext = True, target_size = (size, size),
                                              color_mode = 'rgb', class_mode = 'other',
                                              batch_size = batch_size)
    
    return split_outputs(df_gen, idx)

In [ ]:
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
few_of = lambda aug: iaa.Sometimes(0.2, aug)

seq = iaa.Sequential(
    [
#         apply the following augmenters to most images
#         iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.OneOf([
            sometimes(iaa.CropAndPad(percent=(-0.05, 0.1))), # zoom in
            sometimes(iaa.Affine(scale={"x": (0.6, 1.2), "y": (0.6, 1.2)})) # zoom out
        ]), 
        # execute 1 to 2 of the following augmenters per image
        sometimes(iaa.Affine(translate_px={"x": (0, 25), "y": (0, 25)},
                            rotate = (-25, 25),
                            shear = (-15, 15))), # horizontal/vertical shift
        iaa.SomeOf((1, 4),
                   [iaa.OneOf([
#                        iaa.AllChannelsCLAHE(clip_limit = (0.5, 0.8)),
                       iaa.PerspectiveTransform(scale=(0.01, 0.07)),
                       iaa.Sharpen(alpha=(0, 0.5), lightness=(0.75, 1.5)),
                       iaa.PiecewiseAffine((0.0, 0.01)), # local distortions
                       iaa.GaussianBlur(sigma=(0, 0.7))
                   ]),
                    sometimes(iaa.Dropout((0.005, 0.02), per_channel=0.5)),
                    iaa.AdditiveGaussianNoise(loc=32, scale=0.01*255), # white noise
                    iaa.Add((-20, 50)), # brightness
#                     iaa.AddToHueAndSaturation((-20, 20)),
#                     iaa.AllChannelsHistogramEqualization(),
                    iaa.OneOf([
                        iaa.LinearContrast(alpha=(0.5,1.2), per_channel = True),
#                         iaa.LogContrast(gain=(0.5,0.8), per_channel=True),
#                         iaa.GammaContrast(gamma=(0.4, 1.0)),
                        iaa.ContrastNormalization((0.5, 1.0))
                    ]),
#                     few_of(iaa.Clouds()),
                    few_of(iaa.Multiply((0.7, 1.5), per_channel = True))
            ], random_order=True)
    ], random_order=True)
    

In [ ]:
img_train_gen = ImageDataGenerator(rescale=1/255, preprocessing_function=seq.augment_image)
img_val_gen = ImageDataGenerator(rescale=1/255)
# img_test_gen = ImageDataGenerator(rescale=1/255)

train_gen = read_data(img_train_gen, train_data_path,train_df, idx = split_idx,
                                batch_size = bs, n_classes = tot_num_classes, size = img_size)
val_gen = read_data(img_val_gen,validation_data_path, val_df,idx = split_idx,
                              batch_size = bs,  n_classes = tot_num_classes, size = img_size)
# test_gen = read_data(img_test_gen, test_data_path,test_df,idx = split_idx,
#                                batch_size = bs,  n_classes = tot_num_classes)

In [ ]:
for x, y in train_gen:
    break

print(len(x), len(y), len(y[0]))
print(x[0].shape)
i = 0
y

In [ ]:
fig = plt.figure(figsize=(100, 100))

for i in range(9):
    
    fig.add_subplot(3, 3, i+1)
    
    # Extract corresponding CDC Code
#     actual = [y[0][i], y[1][i], y[2][i], y[3][i], y[4][i], y[5][i]]
    actual = [y[0][i], y[1][i], y[2][i], y[3][i], y[4][i]]
    code = ''
    for j,act in enumerate(actual):
        code = code + str(lb[j].inverse_transform([np.argmax(act)])[0])
                                                   
    plt.imshow(x[i])
    plt.title(code, fontsize = 70)

plt.show()

In [ ]:
#from resnet50 import ResNet50
base_model = ResNet50(include_top=False, weights=None, input_shape = (img_size, img_size, 3))
base_model.load_weights(model_weights_no_top)

In [ ]:
# Freezing base model till first split layer
# trainable = False
# for i,layer in enumerate(base_model.layers[:]):
#     try:
#         if layer.name == model_split_1:
#             trainable = True
#     except:
#         pass
#     layer.trainable = trainable
#     print(layer, layer.trainable)
    
# Freezing entire base model
for layer in base_model.layers[:]:
    layer.trainable = False
    print(layer,layer.trainable)

In [ ]:
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Conv2D
from keras import regularizers

x = base_model.output

x = Conv2D(filters = 512,kernel_size = 3,strides=(1,1), name = 'fc_start')(x)
x = BatchNormalization()(x)

# # Col 2
# x2 = Conv2D(filters = 256,kernel_size = 3,strides=(1,1))(x)
# x2 = GlobalAveragePooling2D()(x2)
# x2 = Dense(1024, activation='relu')(x2)
# x2 = Dropout(0.2)(x2)
# x2 = BatchNormalization()(x2)
# x2 = Dense(512, activation='relu')(x2)
# x2 = Dropout(0.2)(x2)
# out2 = Dense(len(classes_2), activation='softmax', name = 'col2')(x2)

# Col 3
x3 = GlobalAveragePooling2D()(x)
x3 = Dense(1024, activation='relu')(x3)
x3 = Dropout(0.2)(x3)
x3 = BatchNormalization()(x3)
x3 = Dense(512, activation='relu')(x3)
x3 = Dropout(0.2)(x3)
out3 = Dense(len(classes_3), activation='softmax', name = 'col3')(x3)

# Col 4
x4 = Conv2D(filters = 256,kernel_size = 3,strides=(1,1))(x)
x4 = GlobalAveragePooling2D()(x4)
x4 = Dense(1024, activation='relu')(x4)
x4 = Dropout(0.2)(x4)
x4 = BatchNormalization()(x4)
x4 = Dense(512, activation='relu')(x4)
x4 = Dropout(0.2)(x4)
out4 = Dense(len(classes_4), activation='softmax', name = 'col4')(x4)

# Col 5
x5 = GlobalAveragePooling2D()(x)
x5 = Dense(1024, activation='relu')(x5)
x5 = Dropout(0.2)(x5)
x5 = BatchNormalization()(x5)
x5 = Dense(512, activation='relu')(x5)
x5 = Dropout(0.2)(x5)
out5 = Dense(len(classes_5), activation='softmax', name = 'col5')(x5)

# Col 6
x6 = GlobalAveragePooling2D()(x)
x6 = Dense(1024, activation='relu')(x6)
x6 = Dropout(0.2)(x6)
x6 = BatchNormalization()(x6)
x6 = Dense(512, activation='relu')(x6)
x6 = Dropout(0.2)(x6)
out6 = Dense(len(classes_6), activation='softmax', name = 'col6')(x6)

# Col 7
x7 = Conv2D(filters = 256,kernel_size = 3,strides=(1,1))(x)
x7 = GlobalAveragePooling2D()(x7)
x7 = Dense(1024, activation='relu')(x7)
x7 = Dropout(0.2)(x7)
x7 = BatchNormalization()(x7)
x7 = Dense(512, activation='relu')(x7)
x7 = Dropout(0.2)(x7)
out7 = Dense(len(classes_7), activation='softmax', name = 'col7')(x7)

In [ ]:
from keras.models import Model

# model = Model(inputs=base_model.input, outputs=[out2, out3, out4, out5, out6, out7])
model = Model(inputs=base_model.input, outputs=[out3, out4, out5, out6, out7])

In [ ]:
for layer in model.layers:
    print(layer, layer.trainable)

In [ ]:
from keras import backend as K
import tensorflow as tf

def f1(y_true, y_pred):

    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)

    TP = tf.count_nonzero(y_pred * y_true, axis=0, dtype = tf.float64)
    FP = tf.count_nonzero(y_pred * (y_true - 1), axis=0, dtype = tf.float64)
    FN = tf.count_nonzero((y_pred - 1) * y_true, axis=0, dtype = tf.float64)

    add_dummy = lambda x: x + K.epsilon()
    precision = TP /  tf.map_fn(add_dummy, (TP + FP))
    recall = TP / tf.map_fn(add_dummy, (TP + FN))
    f1 = 2 * precision * recall / tf.map_fn(add_dummy, (precision + recall))

    weights = tf.reduce_sum(y_true, axis=0)
    weights /= tf.reduce_sum(weights)
    return tf.reduce_sum(f1 * weights)

In [ ]:
def focal_loss(y_true, y_pred):
    
    gamma=2
    alpha=0.25
    eps = 1e-12
    
    y_pred = K.clip(y_pred, eps, 1.-eps) #improve the stability of the focal loss and see issues 1 for more information

    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

In [ ]:
# Find LR

def lr_finder(mod, train_generator, training_steps, optimizer, loss_type, metrics, lr_range = [1e-5, 0.1]):
    
    lr_find = LR_Find(iterations = training_steps,
                      jump = 6,
                      min_lr = lr_range[0],
                      max_lr = lr_range[1])
    mod.compile(loss = loss_type,
                optimizer=optimizer,
                metrics=metrics)
    mod.fit_generator(train_generator,
                        steps_per_epoch = training_steps,
                        epochs = 1,
                        workers = 1,
                        callbacks = [lr_find])

    fig=plt.figure(figsize=(15, 5))
    fig.add_subplot(1, 2, 1)

    # Show LR vs Loss
    lr_find.plot()
    fig.add_subplot(1, 2, 2)
    
    # Show LR vs Iterations
    lr_find.plot_lr()
    plt.show()
    
    
# lr_finder(mod = model,
#           train_generator = train_gen,
#           training_steps = np.ceil(len(train_ids)/bs),
#           optimizer = Adam(),
#           loss_type = 'categorical_crossentropy',
#           metrics = [f1,"accuracy"])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

if not os.path.exists(save_model_path):
    os.mkdir(save_model_path)

model_checkpoint = ModelCheckpoint(save_model_path + date + '_checkpoint_' + model_name +'.h5',
                                   monitor='val_loss', 
                                   mode = 'auto', save_best_only=True, verbose=2)

tb_cb = callbacks.TensorBoard(log_dir=tf_log_path, histogram_freq=0)

early_stopping = EarlyStopping(monitor='val_loss', mode = 'auto',patience = 5, verbose=2)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', mode = 'auto',factor = 0.8, patience = 3, min_lr = 1e-4, verbose=2)

# For cyclical Learning Rate
sched = LR_Cycle(iterations = np.ceil(len(train_ids)/bs),
                 cycle_mult = 2)

# cbks = [model_checkpoint,early_stopping,reduce_lr]
cbks1 = [model_checkpoint,early_stopping]
cbks2 = [model_checkpoint,early_stopping, sched]

In [ ]:
from keras.optimizers import Adam, SGD

opt = SGD(lr = 2 * 1e-2, momentum = 0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=[f1,"accuracy"])

history = model.fit_generator(train_gen,
                              steps_per_epoch = np.ceil(len(train_ids)/bs),
                              epochs = 2,
                              validation_data = val_gen,
                              validation_steps = np.ceil(len(val_ids)/bs),
                              use_multiprocessing = True,
                              callbacks = cbks1)

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
for layer in model.layers:
    layer.trainable = True
    print(layer, layer.trainable)

In [ ]:
from keras.optimizers import Adam, SGD

# For Differential Learning Rate
split_layer_1 = [layer for layer in model.layers if layer.name == model_split_1][0]
split_layer_2 = [layer for layer in model.layers if layer.name == model_split_2][0]

opt = SGD_dlr(split_l1 = split_layer_1,
               split_l2 = split_layer_2,
               lr = [1e-10, 1e-6, 1e-3],
             momentum = 0.9)

# opt = Adam(lr = 5 * 1e-3)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=[f1,"accuracy"])

# model.compile(loss='categorical_crossentropy',
#               optimizer=opt,
#               metrics=[f1,"accuracy"],
#               loss_weights = loss_wt)

In [ ]:
history = model.fit_generator(train_gen,
                              steps_per_epoch = np.ceil(len(train_ids)/bs),
                              epochs = n_epochs,
                              validation_data = val_gen,
                              validation_steps = np.ceil(len(val_ids)/bs),
                              use_multiprocessing = True,
                              callbacks = cbks2)

# history = model.fit_generator(train_gen,
#                               steps_per_epoch = np.ceil(len(train_ids)/bs),
#                               epochs = n_epochs,
#                               validation_data = val_gen,
#                               validation_steps = np.ceil(len(val_ids)/bs),
#                               use_multiprocessing = True,
#                               callbacks = cbks,
#                               class_weight = cw)

In [ ]:
history.history.keys()

In [ ]:
# sched.plot_lr()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
from keras.models import model_from_json

model_json = model.to_json()
with open(save_model_path + date + '_' + model_name + '_model.json', "w", encoding = 'utf-8') as json_file:
    json_file.write(model_json)
# model.save(save_model_path + date + '_' + model_name + '_model.h5')
model.save_weights(save_model_path + date + '_' + model_name + '_weights.h5')

#Calculate execution time

end = time.time()
dur = end-start

if dur<60:
    print("Execution Time:",dur,"seconds")
elif dur>60 and dur<3600:
    dur=dur/60
    print("Execution Time:",dur,"minutes")
else:
    dur=dur/(60*60)
    print("Execution Time:",dur,"hours")

In [ ]:
model.summary()

In [ ]:
from keras.models import load_model

print(save_model_path + date + '_' + model_name + '_model.json')
json_file = open(save_model_path + date + '_' + model_name + '_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json, 
                        custom_objects={'f1':f1})

model.load_weights(save_model_path + date + '_checkpoint_' + model_name +'.h5')

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=[f1,"accuracy"])

# model.compile(loss='categorical_crossentropy',
#               optimizer=opt,
#               metrics=[f1,"accuracy"],
#               loss_weights = loss_wt)

In [ ]:
def model_evaluation(model, generator, folder, n_steps, n_classes):
    eval_metrics = model.evaluate_generator(generator, workers = 1, steps = n_steps)
    print(folder + ' :')
    print('Loss: ' + str(eval_metrics[0]))
    print('Columnwise f1: ',str([eval_metrics[i] for i in range(1 + n_classes, 1 + 3*n_classes, 2)]))
    print()
    
model_evaluation(model, train_gen, 'Train', n_steps = (len(train_ids)// bs + 1), n_classes = len(classes))
model_evaluation(model, val_gen, 'Validation', n_steps = (len(val_ids)// bs + 1), n_classes = len(classes))
# model_evaluation(model, test_gen, 'Test', n_steps = (len(test_ids)// bs + 1), n_classes = len(classes))

In [ ]:
# Need to add rules for predictions

def get_predictions(model, base_dir, generator, in_df, idx, num_classes, n_steps, threshold = 0.5, size = 512):
    
    y_true = np.split(np.array(in_df.loc[:,list(range(num_classes))]), idx, axis=1)
    print(np.array(in_df.loc[:,list(range(num_classes))]).shape)
    true = [np.argmax(x, axis = 1) for x in y_true]
    
    pred = []
    img_pred = []
    for ind in range(in_df.shape[0]):
        
        images_l = []
        img_predictions = []
    
        img_path = base_dir + in_df.loc[ind, 'filename']
        img = cv2.resize(plt.imread(img_path), (size, size))
        images_l.append(img/255)
        
        for aug in range(4):
            images_l.append(seq.augment_image(img)/255)
        for image in images_l:
            img_predictions.append(model.predict(image.reshape((-1,size, size, 3))))
        predictions = []
        for c in range(len(classes)):
            class_pred = np.array(([(img_predictions[j][c]).reshape(-1) for j in range(len(img_predictions))]))
            predictions.append(np.argmax(np.mean(class_pred, axis = 0)))

        img_pred.append(predictions) 
        pred = [np.array([i[j] for i in img_pred]) for j in range(len(classes))]
    return (true, pred)


print('Train:')
train_true, train_pred = get_predictions(model, train_data_path, train_gen, train_df, idx = split_idx,
                                         num_classes = tot_num_classes, n_steps = (len(train_ids)// bs + 1), size = img_size)
print('\n Validation:')
val_true, val_pred = get_predictions(model, validation_data_path, val_gen, val_df, idx = split_idx,
                                     num_classes = tot_num_classes, n_steps = (len(val_ids)// bs + 1), size = img_size)
# test_true, test_pred = get_predictions(model, test_data_path, test_gen, test_df, idx = split_idx,
#                                        num_classes = tot_num_classes, n_steps = (len(test_ids)// bs + 1))

In [ ]:
def get_predicted_classes(true, pred, lb):
    true_classes = []
    pred_classes = []
    for i in range(len(lb)):
        true_classes.append(lb[i].inverse_transform(true[i]))
        pred_classes.append(lb[i].inverse_transform(pred[i]))
    return true_classes, pred_classes

train_true_classes, train_pred_classes = get_predicted_classes(train_true, train_pred, lb)
val_true_classes, val_pred_classes = get_predicted_classes(val_true, val_pred, lb)

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=False):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(5, 5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
import sklearn.metrics as metrics

def plot_cm(true, pred, classes, i, le):

    report = metrics.confusion_matrix(true, pred, labels = le.classes_)
    plot_confusion_matrix(report,classes, title = 'col_' + str(i))
    
def plot_cm_all_codes(true, pred, list_classes, lb_enc):
    
    for i in range(len(classes)):
        plot_cm(true[i], pred[i], list_classes[i], i+2, lb_enc[i])

In [ ]:
plot_cm_all_codes(train_true_classes, train_pred_classes, classes, lb)

In [ ]:
plot_cm_all_codes(val_true_classes, val_pred_classes, classes, lb)

In [ ]:
def cm_model_report(writer, folder, true, pred, classes):
    
    print('************' + folder + '*************')
    total_samples = len(true[0])
    start = 0
    m = []
    for i in range(len(classes)):
        
        class_true = true[i]
        class_pred = pred[i]
        
        # Accuracy Metrics
        prf = np.round(metrics.precision_recall_fscore_support(class_true, class_pred, average='weighted')[:-1],4)
        acc = np.round(metrics.accuracy_score(class_true, class_pred),4)
        metric_list = ['Col'+str(i+2)]
        metric_list.extend(prf)
        metric_list.extend([acc])
        m.append(metric_list)
        print('col_' + str(i+2) + ': ' + str(metric_list))
        
        # Confusion Matrix
        report = metrics.confusion_matrix(class_true, class_pred, labels = lb[i].classes_)
        df = pd.DataFrame(report, columns = lb[i].classes_, index = lb[i].classes_)
        df.to_excel(writer,sheet_name='CM_' + folder ,startrow = start , startcol=0)   
        
        # Class Scores
        df_class_score = pd.DataFrame(index = lb[i].classes_)
        df_class_score['Precision'] = metrics.precision_score(class_true, class_pred, average = None, labels = lb[i].classes_)
        df_class_score['Recall'] = metrics.recall_score(class_true, class_pred, average = None, labels = lb[i].classes_)
        df_class_score['F1'] = metrics.f1_score(class_true, class_pred, average = None, labels = lb[i].classes_)
        df_class_score.to_excel(writer, sheet_name = folder + '_score', startrow = start, startcol = 0)
    
        start = start + len(classes[i]) + 2
        
    overall_acc = (pd.DataFrame(true) == pd.DataFrame(pred)).all().sum() / total_samples
#     print(folder + ' overall accuracy: ', str(np.round(overall_acc *100, 2)))
    metrics_df = pd.DataFrame(m, columns = ['Col','Precision','Recall','F1','Accuracy'])
    metrics_df.to_excel(writer, sheet_name=folder, index = False)
    pd.DataFrame({'Accuracy' : [overall_acc],'Samples':[total_samples]}).to_excel(writer, sheet_name=folder, startrow = 8, index = False)
    print()
    
    return writer


In [ ]:
print('Overall')
print(save_model_path + date + '_' + model_name + '_report.xlsx')
writer = pd.ExcelWriter(save_model_path + date + '_' + model_name + '_overall_report.xlsx', engine='xlsxwriter')
writer = cm_model_report(writer, 'train', train_true_classes, train_pred_classes, classes)    
writer = cm_model_report(writer, 'val', val_true_classes, val_pred_classes, classes)
# writer = cm_model_report(writer, 'test', test_true, test_pred, classes) 
writer.save()

In [ ]:
def get_type_evaluation(writer, folder, type_crash, true, pred, classes):

    idx = np.where(true[0] == type_crash)[0]
    writer = cm_model_report(writer,
                             folder,
                             list(map(lambda x: x[idx], true)),
                             list(map(lambda x: x[idx], pred)),
                             classes)  
    return writer

In [ ]:
type_crash = 'F'
print(type_crash)
print(save_model_path + date + '_' + model_name + '_' + type_crash + '_report.xlsx')
writer = pd.ExcelWriter(save_model_path + date + '_' + model_name + '_' + type_crash + '_report.xlsx', engine='xlsxwriter')
writer = get_type_evaluation(writer, 'train', type_crash, train_true_classes, train_pred_classes, classes)
writer = get_type_evaluation(writer, 'val', type_crash, val_true_classes, val_pred_classes, classes)
writer.save()

In [ ]:
type_crash = 'L'
print(type_crash)
print(save_model_path + date + '_' + model_name + '_' + type_crash + '_report.xlsx')
writer = pd.ExcelWriter(save_model_path + date + '_' + model_name + '_' + type_crash + '_report.xlsx', engine='xlsxwriter')
writer = get_type_evaluation(writer, 'train', type_crash, train_true_classes, train_pred_classes, classes)
writer = get_type_evaluation(writer, 'val', type_crash, val_true_classes, val_pred_classes, classes)
writer.save()

In [ ]:
type_crash = 'R'
print(type_crash)
print(save_model_path + date + '_' + model_name + '_' + type_crash + '_report.xlsx')
writer = pd.ExcelWriter(save_model_path + date + '_' + model_name + '_' + type_crash + '_report.xlsx', engine='xlsxwriter')
writer = get_type_evaluation(writer, 'train', type_crash, train_true_classes, train_pred_classes, classes)
writer = get_type_evaluation(writer, 'val', type_crash, val_true_classes, val_pred_classes, classes)
writer.save()

In [ ]:
def get_actuals_predictions(split_df, actual_classes, predicted_classes, classes):
    actuals = pd.DataFrame(actual_classes).T
    actuals.columns = ['actual_' + str(i+3) for i in range(len(classes))]
    predicted = pd.DataFrame(predicted_classes).T
    predicted.columns = ['predicted_' + str(i+3) for i in range(len(classes))]
    df = pd.concat([split_df['filename'], actuals, predicted], axis = 1)
    return df

writer = pd.ExcelWriter(save_model_path + date + '_' + model_name + 'actuals_vs_predicted.xlsx', engine='xlsxwriter')
get_actuals_predictions(train_df, train_true_classes, train_pred_classes, classes).to_excel(writer, sheet_name='train')
get_actuals_predictions(val_df, val_true_classes, val_pred_classes, classes).to_excel(writer, sheet_name='val')
writer.save()

In [ ]:
label_enc_dict = {}
for i, c in enumerate(classes):
    label_enc_dict[i] = LabelEncoder().fit(c)
    
val_images = validation_data_path + val_df['filename']
train_images = train_data_path + train_df['filename']

In [ ]:
# Incorrect third column
img_idx = np.where(train_true[0] != train_pred[0])[0]

for i in img_idx:
    
    img_path = train_images[i]
    img = cv2.resize(plt.imread(img_path), (img_size, img_size))/255
    
    img_pred = [train_pred[c][i] for c in range(len(classes))]
    pred_code = [str(label_enc_dict[i].inverse_transform([img_pred[i]])[0]) for i in range(len(img_pred))]
#     pred_code[0] = pred_code[0].zfill(2)
    pred = ''.join(pred_code)

    img_act = [train_true[c][i] for c in range(len(classes))]
    act_code = [str(label_enc_dict[i].inverse_transform([img_act[i]])[0]) for i in range(len(img_act))]
#     act_code[0] = act_code[0].zfill(2)
    act = ''.join(act_code)

    plt.figure(figsize=(8, 8))
    plt.imshow(plt.imread(img_path))
    plt.title(('Actual: ' + act + ' ----- Predicted: ' + pred), fontsize = 20)
    plt.show()


In [ ]:
# Incorrect third column
img_idx = np.where(val_true[0] != val_pred[0])[0]

for i in img_idx:
    
    img_path = val_images[i]
    img = cv2.resize(plt.imread(img_path), (img_size, img_size))/255
    
    img_pred = [val_pred[c][i] for c in range(len(classes))]
    pred_code = [str(label_enc_dict[i].inverse_transform([img_pred[i]])[0]) for i in range(len(img_pred))]
#     pred_code[0] = pred_code[0].zfill(2)
    pred = ''.join(pred_code)

    img_act = [val_true[c][i] for c in range(len(classes))]
    act_code = [str(label_enc_dict[i].inverse_transform([img_act[i]])[0]) for i in range(len(img_act))]
#     act_code[0] = act_code[0].zfill(2)
    act = ''.join(act_code)

    plt.figure(figsize=(8, 8))
    plt.imshow(plt.imread(img_path))
    plt.title(('Actual: ' + act + ' ----- Predicted: ' + pred), fontsize = 20)
    plt.show()
